In [1]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt

In [2]:
import os

def find_Class(directory_path):
    if not os.path.exists(directory_path):
        raise ValueError(f"The directory '{directory_path}' does not exist.")
    if not os.path.isdir(directory_path):
        raise ValueError(f"The path '{directory_path}' is not a directory.")
    
    # Get a list of all entries in the directory
    all_entries = os.listdir(directory_path)
    
    # Filter out only directories
    folders = [entry for entry in all_entries if os.path.isdir(os.path.join(directory_path, entry))]
    
    return folders

In [ ]:
DIRECTORY= r"RAF-DB DATASET/DATASET/train"
CATAGORIES= []
try:
    folders = find_Class(DIRECTORY)
    print(f"Directories in '{DIRECTORY}':")
    for folder in folders:
        CATAGORIES.append(folder)
except ValueError as e:
    print(e)

CATAGORIES
# you can rename these class according to the emotion of the folder

Directories in 'RAF-DB DATASET/DATASET/train':


['1', '2', '3', '4', '5', '6', '7']

In [5]:
data=[]

for categories in CATAGORIES:
    folder=os.path.join(DIRECTORY,categories)
    label=CATAGORIES.index(categories)
    
    
    for img in os.listdir(folder):
        img=os.path.join(folder,img)
        img_arr=cv2.imread(img)
        if img_arr is not None:  # Check if the image is successfully loaded
            img_arr = cv2.resize(img_arr, (100, 100))
            data.append([img_arr, label])
        else:
            print(f"Failed to load image {img}")
            
#         img_arr=cv2.resize(img_arr,(100,100))
        
#         data.append([img_arr,label])

In [6]:
len(data)

12271

### 测试集训练集融合

In [7]:
DIRECTORY= r"RAF-DB DATASET/DATASET/test"
for categories in CATAGORIES:
    folder=os.path.join(DIRECTORY,categories)
    label=CATAGORIES.index(categories)
    
    
    for img in os.listdir(folder):
        img=os.path.join(folder,img)
        img_arr=cv2.imread(img)
        if img_arr is not None:  # Check if the image is successfully loaded
            img_arr = cv2.resize(img_arr, (100, 100))
            data.append([img_arr, label])
        else:
            print(f"Failed to load image {img}")

In [8]:
len(data)

15339

### 混合数据换取更好输出

In [9]:
random.shuffle(data)

In [10]:
x=[]
y=[]


for features,label in data:
    x.append(features)
    y.append(label)

In [11]:
X= np.array(x)
Y=np.array(y)

### 归一化

In [15]:
X=X/255
X.shape

Y.shape

(15339,)

### 创建模型

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
tf.compat.v1.losses.sparse_softmax_cross_entropy

<function tensorflow.python.ops.losses.losses_impl.sparse_softmax_cross_entropy(labels, logits, weights=1.0, scope=None, loss_collection='losses', reduction='weighted_sum_by_nonzero_weights')>

In [18]:
model=Sequential()
model.add( Conv2D(64,(3,3),input_shape=X.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add( Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add( Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(7,activation='softmax'))

In [19]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpoint=ModelCheckpoint(r'fer.keras',
                          monitor='val_accuracy',
                          mode='max',
                          save_best_only=True,
                          verbose=1)
earlystop=EarlyStopping(monitor='val_accuracy',
                        mode='max',
                       min_delta=0.001,
                       patience=20,
                       verbose=1,
                       restore_best_weights=True)

callbacks=[checkpoint,earlystop]

In [20]:
model.fit(X,Y,epochs=50,validation_split=0.25,callbacks =callbacks)

Epoch 1/50


360/360 [==============================] - ETA: 0s - loss: 1.6508 - accuracy: 0.3900
Epoch 1: val_accuracy improved from -inf to 0.38018, saving model to fer.keras
360/360 [==============================] - 42s 115ms/step - loss: 1.6508 - accuracy: 0.3900 - val_loss: 1.6665 - val_accuracy: 0.3802
Epoch 2/50
360/360 [==============================] - ETA: 0s - loss: 1.6386 - accuracy: 0.3911
Epoch 2: val_accuracy did not improve from 0.38018
360/360 [==============================] - 40s 111ms/step - loss: 1.6386 - accuracy: 0.3911 - val_loss: 1.6553 - val_accuracy: 0.3802
Epoch 3/50
360/360 [==============================] - ETA: 0s - loss: 1.6403 - accuracy: 0.3911
Epoch 3: val_accuracy did not improve from 0.38018
360/360 [==============================] - 40s 111ms/step - loss: 1.6403 - accuracy: 0.3911 - val_loss: 1.6518 - val_accuracy: 0.3802
Epoch 4/50
360/360 [==============================] - ETA: 0s - loss: 1.6379 - accuracy: 0.3911
Epoch 4: val_accuracy did not i

KeyboardInterrupt: 

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 32)        0

In [ ]:
from keras.preprocessing import image
import numpy as np

img_pred=image.load_img(r"RAF-DB DATASET/DATASET/test/2/test_0274_aligned.jpg",target_size=(100,100))

img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred, axis=0)


rslt= model.predict(img_pred)

print(rslt)
print(CATAGORIES[np.argmax(rslt)])

FileNotFoundError: [Errno 2] No such file or directory: 'RAF-DB DATASET/DATASET/test/4/test_0274_aligned.jpg'